# Reconnaissance de signatures d'un étudiant parmit un dataset

### Import des librairies

In [1]:
import pygame
from pygame.locals import *;
from math import sin, cos, atan2, sqrt, pi
from xml.dom import minidom
import pandas as pd
import pprint

- pygame : Librairie de jeux vidéo, président ici pour la gesture
- math import sin, cos, atan2, sqrt, pi : Fonctions mathématiques
- xml.dom import minidom : pour naviguer dans le DOM, ici il va servir pour parser
- pandas as pd : Sert ici pour écrire, lire et fusionner des dataFrame ansi que des csv
- numpy as np : Utile ici uniquement pour faire des tableaux
- pprint : pour printer le 1er tableau qui est le dataset de signature sous forme de nuage de point

### Variables super globales à renseigner (1ère lettre en majuscule)

In [2]:
#Taille des signatures, ex : 250 si fenetre de 500 généralement.
Size = 250

### Création d'un jeu de donnée saisie directement de l'utilisateur
#### Les élèves signent chacun leur tour en retenant leur ordre de passage (appuyez sur la croix rouge pour passer à l'étape d'après)

In [3]:
pygame.init();
screen = pygame.display.set_mode((500,500));
pygame.display.flip();
screen.fill((0,0,0))
running = True
recording = False
points = []
signatures = []
while running:
    events = pygame.event.get();
    for e in events:
        if e.type==MOUSEBUTTONDOWN:
            screen.fill((0,0,0))
            points = []
            recording = True
        if e.type==MOUSEBUTTONUP:
            recording = False
            if len(points) > 10:
                signatures.append(points)
        if e.type==MOUSEMOTION:
            if recording:
                mousepos= pygame.mouse.get_pos()
                points.append(list(mousepos))
                pygame.draw.circle(screen,(255,255,255), mousepos, 3);
        if e.type==QUIT:
            running = 0
    pygame.time.wait(2);
    pygame.display.flip();
pp = pprint.PrettyPrinter(indent=2)
print "Jeu de données brut :"
pp.pprint(signatures)

Jeu de données brut :
[ [ [187, 87],
    [186, 88],
    [185, 89],
    [185, 90],
    [184, 91],
    [183, 91],
    [183, 92],
    [182, 93],
    [181, 93],
    [181, 94],
    [180, 95],
    [179, 95],
    [179, 96],
    [178, 97],
    [177, 97],
    [177, 98],
    [176, 98],
    [176, 99],
    [176, 100],
    [175, 101],
    [174, 101],
    [174, 102],
    [174, 104],
    [172, 105],
    [172, 106],
    [172, 107],
    [171, 107],
    [171, 108],
    [171, 109],
    [170, 112],
    [170, 113],
    [170, 114],
    [170, 115],
    [170, 116],
    [170, 117],
    [170, 118],
    [170, 119],
    [170, 122],
    [170, 123],
    [170, 124],
    [170, 125],
    [170, 126],
    [171, 126],
    [171, 127],
    [173, 128],
    [173, 129],
    [174, 130],
    [175, 131],
    [176, 132],
    [176, 133],
    [177, 133],
    [178, 134],
    [179, 135],
    [179, 136],
    [180, 136],
    [181, 136],
    [182, 137],
    [183, 137],
    [184, 138],
    [185, 138],
    [186, 138],
    [187, 139],
    

### Jeu de données de points représentant une signature

In [4]:
class Template:
    def __init__(self, name, points):
        self.points = points
        self.name = name

    def prepare(self):
        self.points = resample(self.points, 64)
        self.points = rotate_to_zero(self.points)
        self.points = scale_to_square(self.points, 250)
        self.points = translate_to_origin(self.points)

templates = []
i = 0
for s in signatures:
    #utile pour le débug : résumer du template
    i = i+1
    templates.append(Template("Signature de l'utilisateur numéro "+`i`, s))

### Fonctions servants à la reconnaissance

In [5]:
def average(xs): return sum(xs) / len(xs)

def resample(points, n):#{{{
    I = pathlength(points) / float(n-1)
    D = 0
    newPoints = [points[0]]
    i = 1
    while i<len(points):
        p_i = points[i]
        d = distance(points[i-1], p_i)
        if (D + d) >= I:
            qx = points[i-1][0] + ((I-D) / d) * (p_i[0] - points[i-1][0])
            qy = points[i-1][1] + ((I-D) / d) * (p_i[1] - points[i-1][1])
            newPoints.append([qx,qy])
            points.insert(i, [qx,qy])
            D = 0
        else: D = D + d
        i+=1
    return newPoints
#}}}

def pathlength(points):#{{{
    d = 0
    for i,p_i in enumerate(points[:len(points)-1]):
        d += distance(p_i, points[i+1])
    return d
#}}}

def distance(p1, p2): return float(sqrt((p1[0] - p2[0])**2 + (p1[1] - p2[1])**2))

def centroid(points): return float(average([float(i[0]) for i in points])), float(average([float(i[1]) for i in points]))

def rotate_to_zero(points):#{{{
    cx, cy = centroid(points)
    theta = atan2(cy - points[0][1], cx - points[0][0])
    newPoints = rotate_by(points, -theta)
    return newPoints
#}}}

def rotate_by(points, theta):#{{{
    cx, cy = centroid(points)
    newpoints = []
    cos_p, sin_p = cos(theta), sin(theta)
    for p in points:
        qx = (p[0] - cx) * cos_p - (p[1] - cy) * sin_p + cx
        qy = (p[0] - cx) * sin_p + (p[1] - cy) * cos_p + cy
        newpoints.append([qx,qy])
    return newpoints
#}}}

def bounding_box(points): #{{{
    minx, maxx = min((p[0] for p in points)), max((p[0] for p in points))
    miny, maxy = min((p[1] for p in points)), max((p[1] for p in points))
    return minx, miny, maxx-minx, maxy - miny
#}}}

def scale_to_square(points, Size):#{{{
    min_x, min_y, w, h = bounding_box(points)
    newPoints = []
    for p in points:
        qx = p[0] * (float(Size) / w )
        qy = p[1] * (float(Size) / h )
        newPoints.append([qx,qy])
    return newPoints
#}}}

def translate_to_origin(points):#{{{
    cx, cy = centroid(points)
    newpoints = []
    for p in points:
        qx, qy = p[0] - cx , p[1] - cy
        newpoints.append([qx,qy])
    return newpoints
#}}}

psi = .5 * (-1 + sqrt(5.))
def distance_at_best_angle(points, T, ta, tb, td):#{{{
    x1 = psi * ta + (1-psi)*tb
    f1 = distance_at_angle(points, T, x1)
    x2 = (1-psi)*ta + psi*tb
    f2 = distance_at_angle(points, T, x2)
    while abs(tb - ta) > td:
        if f1 < f2:
            tb,x2,f2 = x2, x1, f1
            x1 = psi*ta + (1-psi)*tb
            f1 = distance_at_angle(points, T, x1)
        else:
            ta,x1,f1 = x1, x2, f2
            x2 = (1 - psi)*ta + psi*tb
            f2 = distance_at_angle(points, T, x2)
    return min(f1, f2)#}}}

def distance_at_angle(points, T, theta):#{{{
    newpoints = rotate_by(points, theta)
    d = pathdistance(newpoints, T)
    return d
#}}}

def pathdistance(a,b):#{{{
    d = 0
    for ai, bi in zip(a,b):
        d += distance(ai, bi)
    return d / len(a)
#}}}

def classify(candidate_points,templates, screen, debug=False):#{{{
    np = resample(candidate_points,64)
    np = rotate_to_zero(np)
    if debug: plot(np, (255,0,0), screen)
    np = scale_to_square(np, Size)
    if debug: plot(np, (255,0,255), screen)
    np = translate_to_origin(np)
    if debug: plot(np, (0,255,255), screen)
    template, score = recognize(np, templates)
    return template
#}}}
    
def plot(points, color, screen, Size=1):#{{{
    for p in points:
        pygame.draw.circle(screen,color, p, 1);
        pygame.display.flip()
        #}}}

def load_template(xmlfile):#{{{
    points = []
    p_xml = minidom.parse(open(xmlfile,'r'))
    for tag in p_xml.getElementsByTagName("Point"):
        points.append([int(tag.attributes['X'].value), int(tag.attributes['X'].value)])
    return points
#}}}
def recognize(points, templates):#{{{
    b = float("inf")
    #theta = pi / 4 #45 degrees
    theta = 45.
    #t_d = 2 * (pi / 180)
    t_d = 2.
    Tp = None
    num = 0
    for i,T in enumerate(templates):
        Tpoints = T.points
        d = distance_at_best_angle(points, Tpoints, -theta, theta, t_d)
        print "| ", T.name, " | score: ", d
        if d < b:
            b = d
            Tp = T
    return Tp, 1-(b/(0.5 * sqrt(Size*Size * 2)))
#}}}

### Fonctions principale : Lance la fenetre d'enregistrement du nuage de point de la signature à recharcher

In [ ]:
def main():
    for t in templates: t.prepare()
    pygame.init();
    screen = pygame.display.set_mode((500,500));
    pygame.display.flip();
    screen.fill((0,0,0))
    running = True
    recording = False
    points = []
    while running:
        events = pygame.event.get();
        for e in events:
            if e.type==MOUSEBUTTONDOWN:
                screen.fill((0,0,0))
                points = []
                recording = True
            if e.type==MOUSEBUTTONUP:
                recording = False
                if len(points) > 10:
                    template_match = classify(points, templates,screen)
                    if template_match: print "\n La correct est celle de la", template_match.name
            if e.type==MOUSEMOTION:
                if recording:
                    mousepos= pygame.mouse.get_pos()
                    points.append(list(mousepos))
                    pygame.draw.circle(screen,(255,255,255), mousepos, 3);
            if e.type==QUIT:
                running = 0
        pygame.time.wait(5);
        pygame.display.flip();

if __name__=="__main__": main()

|  Signature de l'utilisateur numéro 1  | score:  31.181976134
|  Signature de l'utilisateur numéro 2  | score:  31.1004388177
|  Signature de l'utilisateur numéro 3  | score:  89.3070898898

 La correct est celle de la Signature de l'utilisateur numéro 2


_William GALBY, dans le cadre du cours de Machine Leaning de Jeff Abrahamson - Basé sur l'algo "$1" de Jacob O. Wobbrock, Andrew D. Wilson, Yang Li_